# Ejercicios con RDKit y análisis de toxicidad - EC50 Pimephales

En este notebook trabajaremos con el conjunto de datos generado anteriormente, el cual contiene estructuras químicas (SMILES) y valores de toxicidad frente a *Pimephales promelas* (Fathead Minnow). Usaremos la librería RDKit para procesar las moléculas y obtener descriptores químicos.

**Objetivos:**
- Aprender a sanitizar y preparar estructuras SMILES
- Eliminar duplicados químicos
- Calcular descriptores moleculares usando RDKit
- Explorar relaciones entre descriptores y toxicidad

Importa las librerías necesarias y carga el archivo `'EC50_Pimephales.csv'`. Visualiza las primeras filas para asegurarte que el DataFrame es correcto.

In [4]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 2.2 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
# Escribe aquí tu solución
import pandas as pd 
import numpy as np
from rdkit import Chem

df = pd.read_csv('EC50_Pimephales.csv', sep=';')

### 🔬 Ejercicio 1: Sanitizar SMILES
Convierte cada SMILES en un objeto Mol con RDKit y luego vuelve a generar el SMILES a partir del objeto Mol.

Elimina los compuestos incorrectos.

In [13]:
# Escribe aquí tu solución
# Convertir SMILES a objetos Mol y regenerar SMILES
def sanitize_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return Chem.MolToSmiles(mol)
    except:
        return None
    return None

df['Sanitized_SMILES'] = df['SMILES'].apply(sanitize_smiles)

# Eliminar compuestos incorrectos
df = df[df['Sanitized_SMILES'].notnull()]


### 🧹 Ejercicio 2: Eliminar duplicados
Elimina los SMILES repetidos del DataFrame tras la sanitización.

In [14]:
# Escribe aquí tu solución
df.drop_duplicates(subset=['Sanitized_SMILES'] ).reset_index(drop=True)
print("Number of unique compounds:", len(df['Sanitized_SMILES'].unique()))

Number of unique compounds: 96


### 🧬 Ejercicio 3: Calcular descriptores moleculares
Primero vamos a familiarizarnos con las funciones `GetAtoms()`, `GetSymbol()`, `GetAtomicNum()`, `GetNumAtoms()`, `GetRingInfo()` y `NumRings()`.

Empieza buscando el SMILES del ácido acetilsalicílico y transfórmalo en un objeto `mol`

In [15]:
smi = "CC(=O)OC1=CC=CC=C1C(=O)O"
mol = Chem.MolFromSmiles(smi)

print('Número de átomos en el compuesto --> ', mol.GetNumAtoms())

lista_de_atomos = [atom.GetSymbol() for atom in mol.GetAtoms()]
print('Lista de los átomos en el compuesto --> ', lista_de_atomos)

numeros_atomicos = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
print('Lista de los números atómicos --> ', numeros_atomicos)

numero_anillos = mol.GetRingInfo().NumRings()
print('Número de anillos en el compuesto --> ', numero_anillos)

Número de átomos en el compuesto -->  13
Lista de los átomos en el compuesto -->  ['C', 'C', 'O', 'O', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'O']
Lista de los números atómicos -->  [6, 6, 8, 8, 6, 6, 6, 6, 6, 6, 6, 8, 8]
Número de anillos en el compuesto -->  1


Ahora define una función que calcule el número de átomos de nitrogeno en un compuesto.

In [19]:
# Escribe aquí tu solución
def num_nitrogens(smi):
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        return sum(1 for atom in mol.GetAtoms() if atom.GetSymbol() == 'N')
    return 0

Ahora una que calcule la proporción de oxígeno en un compuesto.

In [16]:
def oxigen_proportion(smi):
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        num_oxigen = sum(1 for atom in mol.GetAtoms() if atom.GetSymbol() == 'O')
        return num_oxigen / mol.GetNumAtoms()
    return 0

Y por último una que calcule la cantidad de anillos aromáticos en un compuesto.

In [17]:
# Escribe aquí tu solución
def num_aromatic_rings(smi):
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        return len(mol.GetRingInfo().BondRings())
    return 0

Aplica las funciones que has definido al ácido acetilsalicílico

In [ ]:
# Escribe aquí tu solución
smi_ac_acid = "CC(=O)OC1=CC=CC=C1C(=O)O"
num_nitrogens_ac_acid = num_nitrogens(smi_ac_acid)
oxigen_proportion_ac_acid = oxigen_proportion(smi_ac_acid)
num_aromatic_rings_ac_acid = num_aromatic_rings(smi_ac_acid)

print(f"El número de átomos de nitrógeno en el ácido acético es: {num_nitrogens_ac_acid}")
print(f"La proporción de oxígeno en el ácido acético es: {oxigen_proportion_ac_acid:.2f}")
print(f"El número de anillos aromáticos en el ácido acético es: {num_aromatic_rings_ac_acid}")

# Use the functions from above and crea

El número de átomos de nitrógeno en el ácido acético es: 0
La proporción de oxígeno en el ácido acético es: 0.31
El número de anillos aromáticos en el ácido acético es: 1


Ahora usa las funciones con el DataFrame y crea 3 columnas nuevas, una por cada descriptor.

In [ ]:
# Escribe aquí tu solución


### 📊 Ejercicio 4: Estadísticas y visualización
Muestra estadísticas de los descriptores y crea histogramas.

In [23]:
#show descriptors about df data
df.describe()

,Conc 1 Mean (Standardized),log_Conc
count,590.000000,590.000000
mean,368.747772,0.277976
std,3733.425665,1.359091
min,0.000200,-3.698970
25%,0.170250,-0.768948
50%,1.500000,0.176091
75%,13.675000,1.135547
max,62000.000000,4.792392


In [24]:
df.head()

,CAS Number,Chemical Name,SMILES,Species Scientific Name,Species Common Name,Species Sub Phylum,Effect,Organism Lifestage,Exposure Duration Mean (Days),Conc 1 Mean (Standardized),Conc 1 Mean Op (Standardized),Conc 1 Units (Standardized),log_Conc,Es tóxico,Sanitized_SMILES
0,50-29-3,"1,1'-(2,2,2-Trichloroethylidene)bis[4-chlorobe...",Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,Pimephales promelas,Fathead Minnow,Vertebrata,Mortality,Not reported,NR,0.0138,NaN,AI mg/L,-1.860121,Tóxico,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1
1,50-29-3,"1,1'-(2,2,2-Trichloroethylidene)bis[4-chlorobe...",Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,Pimephales promelas,Fathead Minnow,Vertebrata,Mortality,Not reported,NR,0.0200,>,AI mg/L,-1.698970,Tóxico,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1
2,50-29-3,"1,1'-(2,2,2-Trichloroethylidene)bis[4-chlorobe...",Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,Pimephales promelas,Fathead Minnow,Vertebrata,Mortality,Not reported,NR,0.0142,NaN,AI mg/L,-1.847712,Tóxico,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1
3,50-29-3,"1,1'-(2,2,2-Trichloroethylidene)bis[4-chlorobe...",Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,Pimephales promelas,Fathead Minnow,Vertebrata,Mortality,Not reported,NR,0.0099,NaN,AI mg/L,-2.004365,Tóxico,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1
4,50-29-3,"1,1'-(2,2,2-Trichloroethylidene)bis[4-chlorobe...",Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,Pimephales promelas,Fathead Minnow,Vertebrata,Mortality,Not reported,NR,0.0124,NaN,AI mg/L,-1.906578,Tóxico,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1


### 📈 Ejercicio 5: Relación con toxicidad
Representa gráficos de dispersión entre cada descriptor y el valor logarítmico de concentración.

In [ ]:
# Escribe aquí tu solución

### 💾 Ejercicio 6: Guardar dataset final
Guarda el DataFrame con SMILES sanitizados y descriptores en un nuevo CSV.

In [ ]:
# Escribe aquí tu solución